In [1]:
import cv2
import numpy as np
import onnxruntime

# 读取两张图像
image1 = cv2.imread("../datas/motion1.png")
image2 = cv2.imread("../datas/motion2.png")

# 转换成RGB格式
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

# 获取图像的高度和宽度
height, width, _ = image1.shape

# 定义一个空白图像，用于存放变换后的图像
warped_image = np.zeros_like(image1)

# 定义一个空白图像，用于存放遮挡区域的掩码
occlusion_mask = np.zeros((height, width), dtype=np.uint8)

# 加载光流模型
onnx_file = "GMFlowNet.onnx"
infer_task = onnxruntime.InferenceSession(onnx_file, providers=['CPUExecutionProvider'])

# 将图像转换成张量格式，即B x C x H x W
def convert_to_tensor(x):
    x_tensor = x.transpose(2, 0, 1)
    x_tensor = np.ascontiguousarray(x_tensor)
    x_tensor = np.expand_dims(x_tensor, axis=0)
    return x_tensor.astype("float32")

# 计算前向光流，即I_1 -> I_2 的光流
[forward_flow] = infer_task.run(
    ["flow"],
    {"image1": convert_to_tensor(image1), "image2": convert_to_tensor(image2)}
)

# 将光流转换成数组格式，即H x W x C
forward_flow = np.ascontiguousarray(forward_flow[0].transpose(1, 2, 0))

# 定义一个函数，用于将坐标限制在有效范围内
def clip(x, low, high):
    return min(max(x, low), high)

# 定义一个函数，用于将小数坐标四舍五入到最近的整数坐标
def nearest_round(x):
    return int(round(x))

# 遍历每个像素点，根据光流向量进行变换和遮挡检测
for i in range(height):
    for j in range(width):
        # 获取当前坐标对应的光流值
        u = forward_flow[i, j, 0]
        v = forward_flow[i, j, 1]

        # 找到目标坐标，并截断在有效区域内
        x = clip(nearest_round(i + v), 0, height - 1)
        y = clip(nearest_round(j + u), 0, width - 1)

        # 如果目标位置已经有值，说明存在遮挡或者运动不连续，设置遮挡掩码为255
        if warped_image[x, y].any():
            occlusion_mask[x, y] = 255

        # 将原始图像的当前位置的值赋给变换后的图像的目标位置
        warped_image[x, y] = image1[i, j]

# 对变换后的图像进行插值，填补孔洞
warped_image = cv2.inpaint(warped_image, occlusion_mask, 3, cv2.INPAINT_TELEA)

# 显示结果
cv2.imshow("Image 1", image1)
cv2.imshow("Image 2", image2)
cv2.imshow('warped_image',warped_image)

NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from GMFlowNet.onnx failed:Load model GMFlowNet.onnx failed. File doesn't exist